In [1]:
import mlflow
mlflow.set_tracking_uri("file:../mlruns")

import pandas as pd

# Load MLflow experiment runs
experiment_name = "customer_churn_experiments"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
runs.head()

c:\Users\jahid\Desktop\New folder\Customer-Churn\.venv\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall,metrics.precision,metrics.accuracy,metrics.roc_auc,...,params.subsample,params.n_estimators,params.max_depth,params.colsample_bytree,params.learning_rate,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.runName
0,33b0d8f70cfb4f0d81aef5cc1ce4d1cd,396059338896050123,FINISHED,file:///C:/Users/jahid/Desktop/New folder/Cust...,2026-02-18 20:16:10.662000+00:00,2026-02-18 20:16:10.856000+00:00,0.668449,0.568182,0.777147,0.842135,...,None,None,None,None,None,jahid,C:\Users\jahid\Desktop\New folder\Customer-Chu...,LOCAL,282b7536fd27a4ea5a4dbe9d3cd73662c646d722,efficient-shark-621
1,bee36697c72947a9b61864039cd24c7b,396059338896050123,FINISHED,file:///C:/Users/jahid/Desktop/New folder/Cust...,2026-02-18 20:14:45.075000+00:00,2026-02-18 20:15:08.010000+00:00,0.663102,0.590476,0.788502,0.847214,...,0.8828492734716423,101,3,0.7010731254820163,0.0718074803798502,jahid,C:\Users\jahid\Desktop\New folder\Customer-Chu...,LOCAL,282b7536fd27a4ea5a4dbe9d3cd73662c646d722,brawny-bass-275


In [2]:
metrics_to_compare = [
    "metrics.roc_auc",
    "metrics.precision",
    "metrics.recall",
    "metrics.f1",
    "metrics.pr_auc",
]

comparison = (
    runs
    .dropna(subset=["params.model_type"])
    .groupby("params.model_type")[metrics_to_compare]
    .mean()
    .sort_values("metrics.roc_auc", ascending=False)
)

comparison

,metrics.roc_auc,metrics.precision,metrics.recall,metrics.f1,metrics.pr_auc
params.model_type,,,,,
xgboost,0.847214,0.590476,0.663102,0.624685,0.662360
logistic,0.842135,0.568182,0.668449,0.614251,0.634339


In [3]:
#Best Performing Runs
runs.sort_values("metrics.roc_auc", ascending=False).head(5)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall,metrics.precision,metrics.accuracy,metrics.roc_auc,...,params.subsample,params.n_estimators,params.max_depth,params.colsample_bytree,params.learning_rate,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.runName
1,bee36697c72947a9b61864039cd24c7b,396059338896050123,FINISHED,file:///C:/Users/jahid/Desktop/New folder/Cust...,2026-02-18 20:14:45.075000+00:00,2026-02-18 20:15:08.010000+00:00,0.663102,0.590476,0.788502,0.847214,...,0.8828492734716423,101,3,0.7010731254820163,0.0718074803798502,jahid,C:\Users\jahid\Desktop\New folder\Customer-Chu...,LOCAL,282b7536fd27a4ea5a4dbe9d3cd73662c646d722,brawny-bass-275
0,33b0d8f70cfb4f0d81aef5cc1ce4d1cd,396059338896050123,FINISHED,file:///C:/Users/jahid/Desktop/New folder/Cust...,2026-02-18 20:16:10.662000+00:00,2026-02-18 20:16:10.856000+00:00,0.668449,0.568182,0.777147,0.842135,...,None,None,None,None,None,jahid,C:\Users\jahid\Desktop\New folder\Customer-Chu...,LOCAL,282b7536fd27a4ea5a4dbe9d3cd73662c646d722,efficient-shark-621


## Best Runs — Interpretation

**Top Runs Summary:**

| Strategy | ROC-AUC | Recall | F1 | PR-AUC |
|---|---|---|---|---|
| XGBoost | 0.847214 | 0.663 | 0.6247 | 0.6624 |
| Logistic Regression | 0.842135 | 0.668 | 0.6143 | 0.6343 |


**Final Model Decision:**
- XGBoost selected with decision threshold tuned to **0.4**
- Threshold 0.4 boosts recall from ~54% (default 0.5) to ~68% — catching 50+ more real churners
- Trade-off accepted: ~65 additional false positives offset by catching significantly more churners
- Business rationale: cost of missed churner (lost revenue) >> cost of false positive (retention offer)